In [1]:
# Import libraries

import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



import tensorflow_datasets as tfds

In [2]:
# Load the data

p_camelyon, p_camelyon_info = tfds.load("patch_camelyon", with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/patch_camelyon/2.0.0.incompleteEEHBDX/patch_camelyon-test.tfrecord


...

  0%|          | 0/262144 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/patch_camelyon/2.0.0.incompleteEEHBDX/patch_camelyon-validation.tfrecord


  0%|          | 0/32768 [00:00<?, ? examples/s]

Dataset patch_camelyon downloaded and prepared to /root/tensorflow_datasets/patch_camelyon/2.0.0. Subsequent calls will reuse this data.


In [3]:
# Defining a "normalize" function to process the data before feeding it into the deep neural network. 

def normalize(x):
  image, label = x['image'], x['label']
  image = tf.image.convert_image_dtype(image, tf.float32)
  label = tf.one_hot(label, 2, dtype=tf.float32)
  return image, label

In [68]:
# Applying "normalize" function along with others to the data

# Processing the training dataset
train_ds = p_camelyon['train'].map(normalize, num_parallel_calls=8)
train_ds = train_ds.shuffle(1024)
train_ds = train_ds.repeat()
train_ds = train_ds.batch(64)
train_ds = train_ds.prefetch(2)

# Processing validation dataset
validation_ds = p_camelyon['validation'].map(normalize, num_parallel_calls=8)
validation_ds = validation_ds.repeat()
validation_ds = validation_ds.batch(128)
validation_ds = validation_ds.prefetch(2)

#Processing the test dataset
test_ds = p_camelyon['test'].map(normalize, num_parallel_calls=8)
test_ds = test_ds.batch(128)
test_ds = test_ds.prefetch(2)

#Seperating image and label into different variables
train_images, train_labels = next(iter(train_ds))
valid_images, valid_labels = next(iter(validation_ds))
test_images, test_labels  = next(iter(test_ds))


In [69]:
import math

batch_size_for_training = 64
batch_size_for_validation = 128
batch_size_for_test = 128

training_size = 262144
validation_size = 32768
test_size =  32768
        

# Calculate steps for training and testing the model
calculate_steps_training = lambda x: int(math.ceil(1. * x / batch_size_for_training))
calculate_steps_validation = lambda x: int(math.ceil(1. * x / batch_size_for_validation))
calculate_steps_test= lambda x: int(math.ceil(1. * x / batch_size_for_test))

steps_per_epoch = calculate_steps_training(training_size) 
validation_steps = calculate_steps_validation(validation_size)
steps = calculate_steps_test(test_size)

print("Training steps: ", steps_per_epoch)
print("Validation steps: ", validation_steps)
print("Testing steps: ", steps)


Training steps:  4096
Validation steps:  256
Testing steps:  256


In [8]:
import matplotlib.pyplot as plt

def plot_hist(history):
  plt.plot(history.history["acc"])
  plt.plot(history.history["val_acc"])
  plt.title("Model Accuracy")
  plt.ylabel("Accuracy")
  plt.xlabel("Epochs")
  plt.legend(["train", "validation"], loc="upper left")
  plt.show()

Models

Models 1, 2, 3

In [19]:
# Building the Convolutonal Neural Network Version 2 Model 1

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Rely activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = binary_cross_entropy
model_1_version_2 = Model(inputs=input, outputs = output)
model_1_version_2.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['acc'])

model_1_version_2.summary()
#plot_model(model_1_version_2)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 94, 94, 16)        448       
                                                                 
 conv2d_4 (Conv2D)           (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 46, 46, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 44, 44, 32)        4640      
                                                                 
 conv2d_6 (Conv2D)           (None, 42, 42, 16)        4624      
                                                           

In [20]:
# Building the Convolutonal Neural Network Version 2 Model 2

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Rely activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = binary_cross_entropy
model_2_version_2 = Model(inputs=input, outputs = output)
model_2_version_2.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['acc'])

model_2_version_2.summary()
#plot_model(model_2_version_2)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 94, 94, 16)        448       
                                                                 
 conv2d_10 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 46, 46, 16)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_12 (Conv2D)          (None, 42, 42, 16)        4624      
                                                           

In [21]:
# Building the Convolutonal Neural Network Version 2 Model 3

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Rely activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = binary_cross_entropy
model_3_version_2 = Model(inputs=input, outputs = output)
model_3_version_2.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['acc'])

model_3_version_2.summary()
#plot_model(model_3_version_2)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_16 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 46, 46, 16)       0         
 2D)                                                             
                                                                 
 conv2d_17 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_18 (Conv2D)          (None, 42, 42, 16)        4624      
                                                           

Models 1.1, 1.2, 2.1, 2.2, 3.1, 3.2

In [31]:
# Building the Convolutonal Nueral Network Model 1.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = hinge
model_1_1_version_2 = Model(inputs=input, outputs = output)
model_1_1_version_2.compile(optimizer='adam', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_1_1_version_2.summary()
#plot_model(model_1_1_version_2)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_22 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_24 (Conv2D)          (None, 42, 42, 16)        4624      
                                                           

In [32]:
# Building the Convolutonal Nueral Network Model 1.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function =   MSE
model_1_2_version_2 = Model(inputs=input, outputs = output)
model_1_2_version_2.compile(optimizer='adam', 
              loss='MSE',
              metrics=['acc'])

model_1_2_version_2.summary()
#plot_model(model_1_2_version_2)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_27 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_28 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_30 (Conv2D)          (None, 42, 42, 16)        4624      
                                                           

In [33]:
# Building the Convolutonal Nueral Network Model 2.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Sigmoid activation functions only with softmax for the last Dense layer

x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = hinge
model_2_1_version_2 = Model(inputs=input, outputs = output)
model_2_1_version_2.compile(optimizer='adam', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_2_1_version_2.summary()
#plot_model(model_2_2_version_2)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_33 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_34 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_35 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_36 (Conv2D)          (None, 42, 42, 16)        4624      
                                                           

In [34]:
# Building the Convolutonal Nueral Network Model 2.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Sigmoid activation functions only with softmax for the last Dense layer

x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = MSE
model_2_2_version_2 = Model(inputs=input, outputs = output)
model_2_2_version_2.compile(optimizer='adam', 
              loss='MSE',
              metrics=['acc'])

model_2_2_version_2.summary()
#plot_model(model_2_2_version_2)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_39 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_40 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_42 (Conv2D)          (None, 42, 42, 16)        4624      
                                                           

In [35]:
# Building the Convolutonal Neural Network Model 3.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# A conbination of relu and sigmoid activation functions
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = hinge
model_3_1_version_2 = Model(inputs=input, outputs = output)
model_3_1_version_2.compile(optimizer='adam', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_3_1_version_2.summary()
#plot_model(model_3_1_version_2)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_45 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_46 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_47 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_48 (Conv2D)          (None, 42, 42, 32)        9248      
                                                           

In [36]:
# Building the Convolutonal Neural Network Model 3.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# A conbination of relu and sigmoid activation functions
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = MSE
model_3_2_version_2 = Model(inputs=input, outputs = output)
model_3_2_version_2.compile(optimizer='adam', 
              loss='MSE',
              metrics=['acc'])

model_3_2_version_2.summary()
#plot_model(model_3_2_version_2)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_51 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_52 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_53 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_54 (Conv2D)          (None, 42, 42, 32)        9248      
                                                           

Models 1.1.1, 1.1.2, 1.1.3;
1.2.1, 1.2.2, 1.2.3, 2.1.1, 2.1.2, 2.1.3, 2.2.1, 2.2.2, 2.2.3, 3.1.1, 3.1.2, 3.1.3, 3.2.1, 3.2.2, 3.2.3.

In [45]:
# Building the Convolutonal Nueral Network Model 1.1.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = sgd
# Loss function = hinge
model_1_1_1_version_2 = Model(inputs=input, outputs = output)
model_1_1_1_version_2.compile(optimizer='sgd', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_1_1_1_version_2.summary()
#plot_model(model_1_1_1_version_2)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_57 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_58 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_59 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_60 (Conv2D)          (None, 42, 42, 16)        4624      
                                                          

In [46]:
# Building the Convolutonal Nueral Network Model 1.1.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adagrad
# Loss function = hinge
model_1_1_2_version_2 = Model(inputs=input, outputs = output)
model_1_1_2_version_2.compile(optimizer='adagrad', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_1_1_2_version_2.summary()
#plot_model(model_1_1_2_version_2)

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_63 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_64 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_65 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_66 (Conv2D)          (None, 42, 42, 16)        4624      
                                                          

In [48]:
# Building the Convolutonal Nueral Network Model 1.1.3 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = RMSprop
# Loss function = hinge
model_1_1_3_version_2 = Model(inputs=input, outputs = output)
model_1_1_3_version_2.compile(optimizer='RMSprop', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_1_1_3_version_2.summary()
#plot_model(model_1_1_3_version_2)

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_75 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_76 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_77 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_78 (Conv2D)          (None, 42, 42, 16)        4624      
                                                          

In [49]:
# Building the Convolutonal Nueral Network Model 1.2.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = sgd
# Loss function =   MSE
model_1_2_1_version_2 = Model(inputs=input, outputs = output)
model_1_2_1_version_2.compile(optimizer='sgd', 
              loss='MSE',
              metrics=['acc'])

model_1_2_1_version_2.summary()
#plot_model(model_1_2_1_version_2)

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_81 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_82 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_83 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_84 (Conv2D)          (None, 42, 42, 16)        4624      
                                                          

In [50]:
# Building the Convolutonal Nueral Network Model 1.2.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adagrad
# Loss function =   MSE
model_1_2_2_version_2 = Model(inputs=input, outputs = output)
model_1_2_2_version_2.compile(optimizer='adagrad', 
              loss='MSE',
              metrics=['acc'])

model_1_2_2_version_2.summary()
#plot_model(model_1_2_2_version_2)

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_87 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_88 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_89 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_90 (Conv2D)          (None, 42, 42, 16)        4624      
                                                          

In [51]:
# Building the Convolutonal Nueral Network Model 1.2.3 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = RMSprop
# Loss function =   MSE
model_1_2_3_version_2 = Model(inputs=input, outputs = output)
model_1_2_3_version_2.compile(optimizer='RMSprop', 
              loss='MSE',
              metrics=['acc'])

model_1_2_3_version_2.summary()
#plot_model(model_1_2_3_version_2)

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_93 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_94 (Conv2D)          (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_95 (Conv2D)          (None, 44, 44, 32)        4640      
                                                                 
 conv2d_96 (Conv2D)          (None, 42, 42, 16)        4624      
                                                          

In [52]:
# Building the Convolutonal Nueral Network Model 2.1.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Sigmoid activation functions only with softmax for the last Dense layer

x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = sgd
# Loss function = hinge
model_2_1_1_version_2 = Model(inputs=input, outputs = output)
model_2_1_1_version_2.compile(optimizer='sgd', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_2_1_1_version_2.summary()
#plot_model(model_2_1_1_version_2)

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_99 (Conv2D)          (None, 94, 94, 16)        448       
                                                                 
 conv2d_100 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_51 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_101 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_102 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [53]:
# Building the Convolutonal Nueral Network Model 2.1.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Sigmoid activation functions only with softmax for the last Dense layer

x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adagrad
# Loss function = hinge
model_2_1_2_version_2 = Model(inputs=input, outputs = output)
model_2_1_2_version_2.compile(optimizer='adagrad', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_2_1_2_version_2.summary()
#plot_model(model_2_1_2_version_2)

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_105 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_106 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_54 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_107 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_108 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [54]:
# Building the Convolutonal Nueral Network Model 2.1.3 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Sigmoid activation functions only with softmax for the last Dense layer

x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = RMSprop
# Loss function = hinge
model_2_1_3_version_2 = Model(inputs=input, outputs = output)
model_2_1_3_version_2.compile(optimizer='RMSprop', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_2_1_3_version_2.summary()
#plot_model(model_2_1_3_version_2)

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_111 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_112 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_113 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_114 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [55]:
# Building the Convolutonal Nueral Network Model 2.2.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Sigmoid activation functions only with softmax for the last Dense layer

x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = sgd
# Loss function = MSE
model_2_2_1_version_2 = Model(inputs=input, outputs = output)
model_2_2_1_version_2.compile(optimizer='sgd', 
              loss='MSE',
              metrics=['acc'])

model_2_2_1_version_2.summary()
#plot_model(model_2_2_1_version_2)

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_117 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_118 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_119 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_120 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [56]:
# Building the Convolutonal Nueral Network Model 2.2.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Sigmoid activation functions only with softmax for the last Dense layer

x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adagrad
# Loss function = MSE
model_2_2_2_version_2 = Model(inputs=input, outputs = output)
model_2_2_2_version_2.compile(optimizer='adagrad', 
              loss='MSE',
              metrics=['acc'])

model_2_2_2_version_2.summary()
#plot_model(model_2_2_2_version_2)

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_123 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_124 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_63 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_125 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_126 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [57]:
# Building the Convolutonal Nueral Network Model 2.2.3 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Sigmoid activation functions only with softmax for the last Dense layer

x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = RMSprop
# Loss function = MSE
model_2_2_3_version_2 = Model(inputs=input, outputs = output)
model_2_2_3_version_2.compile(optimizer='RMSprop', 
              loss='MSE',
              metrics=['acc'])

model_2_2_3_version_2.summary()
#plot_model(model_2_2_3_version_2)

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_129 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_130 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_66 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_131 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_132 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [58]:
# Building the Convolutonal Neural Network Model 3.1.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# A conbination of relu and sigmoid activation functions
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = sgd
# Loss function = hinge
model_3_1_1_version_2 = Model(inputs=input, outputs = output)
model_3_1_1_version_2.compile(optimizer='sgd', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_3_1_1_version_2.summary()
#plot_model(model_3_1_1_version_2)

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_135 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_136 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_69 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_137 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_138 (Conv2D)         (None, 42, 42, 32)        9248      
                                                          

In [59]:
# Building the Convolutonal Neural Network Model 3.1.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# A conbination of relu and sigmoid activation functions
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adagrad
# Loss function = hinge
model_3_1_2_version_2 = Model(inputs=input, outputs = output)
model_3_1_2_version_2.compile(optimizer='adagrad', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_3_1_2_version_2.summary()
#plot_model(model_3_1_2_version_2)

Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_141 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_142 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_72 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_143 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_144 (Conv2D)         (None, 42, 42, 32)        9248      
                                                          

In [61]:
# Building the Convolutonal Neural Network Model 3.1.3 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# A conbination of relu and sigmoid activation functions
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = RMSprop
# Loss function = hinge
model_3_1_3_version_2 = Model(inputs=input, outputs = output)
model_3_1_3_version_2.compile(optimizer='RMSprop', 
              loss=tf.keras.losses.Hinge(),
              metrics=['acc'])

model_3_1_3_version_2.summary()
#plot_model(model_3_1_3_version_2)

Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_153 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_154 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_78 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_155 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_156 (Conv2D)         (None, 42, 42, 32)        9248      
                                                          

In [60]:
# Building the Convolutonal Neural Network Model 3.2.1 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# A conbination of relu and sigmoid activation functions
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = sgd
# Loss function = MSE
model_3_2_1_version_2 = Model(inputs=input, outputs = output)
model_3_2_1_version_2.compile(optimizer='sgd', 
              loss='MSE',
              metrics=['acc'])

model_3_2_1_version_2.summary()
#plot_model(model_3_2_1_version_2)

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_147 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_148 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_75 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_149 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_150 (Conv2D)         (None, 42, 42, 32)        9248      
                                                          

In [62]:
# Building the Convolutonal Neural Network Model 3.2.2 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# A conbination of relu and sigmoid activation functions
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adagrad
# Loss function = MSE
model_3_2_2_version_2 = Model(inputs=input, outputs = output)
model_3_2_2_version_2.compile(optimizer='adagrad', 
              loss='MSE',
              metrics=['acc'])

model_3_2_2_version_2.summary()
#plot_model(model_3_2_2_version_2)

Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_159 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_160 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_81 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_161 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_162 (Conv2D)         (None, 42, 42, 32)        9248      
                                                          

In [63]:
# Building the Convolutonal Neural Network Model 3.2.3 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# A conbination of relu and sigmoid activation functions
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(input)
x = Conv2D(16, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = Conv2D(64, (3,3), activation='sigmoid', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(rate=0.2)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = RMSprop
# Loss function = MSE
model_3_2_3_version_2 = Model(inputs=input, outputs = output)
model_3_2_3_version_2.compile(optimizer='RMSprop', 
              loss='MSE',
              metrics=['acc'])

model_3_2_3_version_2.summary()
#plot_model(model_3_2_3_version_2)

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_165 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_166 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_84 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_167 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_168 (Conv2D)         (None, 42, 42, 32)        9248      
                                                          

In [70]:
history_1_1_1_version_2= model_1_1_1_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


Epoch 1/10
4096/4096 [==============================] - 97s 24ms/step - loss: 0.7062 - acc: 0.7948 - val_loss: 0.7093 - val_acc: 0.7913
Epoch 2/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.6908 - acc: 0.8103 - val_loss: 0.6966 - val_acc: 0.8034
Epoch 3/10
4096/4096 [==============================] - 90s 22ms/step - loss: 0.6820 - acc: 0.8188 - val_loss: 0.6872 - val_acc: 0.8133
Epoch 4/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.6750 - acc: 0.8258 - val_loss: 0.6859 - val_acc: 0.8165
Epoch 5/10
4096/4096 [==============================] - 90s 22ms/step - loss: 0.6679 - acc: 0.8329 - val_loss: 0.6810 - val_acc: 0.8190
Epoch 6/10
4096/4096 [==============================] - 90s 22ms/step - loss: 0.6620 - acc: 0.8386 - val_loss: 0.6847 - val_acc: 0.8168
Epoch 7/10
4096/4096 [==============================] - 91s 22ms/step - loss: 0.6577 - acc: 0.8428 - val_loss: 0.6849 - val_acc: 0.8155
Epoch 8/10
4096/4096 [==========================

In [71]:
history_1_1_2_version_2= model_1_1_2_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


Epoch 1/10
4096/4096 [==============================] - 92s 22ms/step - loss: 0.9667 - acc: 0.5397 - val_loss: 0.8133 - val_acc: 0.7005
Epoch 2/10
4096/4096 [==============================] - 88s 22ms/step - loss: 0.7583 - acc: 0.7492 - val_loss: 0.7642 - val_acc: 0.7378
Epoch 3/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.7383 - acc: 0.7644 - val_loss: 0.7616 - val_acc: 0.7400
Epoch 4/10
4096/4096 [==============================] - 110s 27ms/step - loss: 0.7338 - acc: 0.7683 - val_loss: 0.7625 - val_acc: 0.7378
Epoch 5/10
4096/4096 [==============================] - 90s 22ms/step - loss: 0.7303 - acc: 0.7714 - val_loss: 0.7582 - val_acc: 0.7428
Epoch 6/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.7281 - acc: 0.7734 - val_loss: 0.7551 - val_acc: 0.7456
Epoch 7/10
4096/4096 [==============================] - 90s 22ms/step - loss: 0.7265 - acc: 0.7750 - val_loss: 0.7580 - val_acc: 0.7426
Epoch 8/10
4096/4096 [=========================

In [72]:
history_1_2_1_version_2= model_1_2_1_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


Epoch 1/10
4096/4096 [==============================] - 91s 22ms/step - loss: 0.1765 - acc: 0.7390 - val_loss: 0.1485 - val_acc: 0.7856
Epoch 2/10
4096/4096 [==============================] - 108s 26ms/step - loss: 0.1443 - acc: 0.7978 - val_loss: 0.1531 - val_acc: 0.7844
Epoch 3/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.1305 - acc: 0.8194 - val_loss: 0.1338 - val_acc: 0.8081
Epoch 4/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.1231 - acc: 0.8305 - val_loss: 0.1580 - val_acc: 0.7691
Epoch 5/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.1176 - acc: 0.8383 - val_loss: 0.1399 - val_acc: 0.8057
Epoch 6/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.1126 - acc: 0.8457 - val_loss: 0.1273 - val_acc: 0.8214
Epoch 7/10
4096/4096 [==============================] - 88s 22ms/step - loss: 0.1080 - acc: 0.8519 - val_loss: 0.1166 - val_acc: 0.8369
Epoch 8/10
4096/4096 [=========================

In [73]:
history_1_2_2_version_2= model_1_2_2_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


Epoch 1/10
4096/4096 [==============================] - 92s 22ms/step - loss: 0.2449 - acc: 0.5876 - val_loss: 0.2134 - val_acc: 0.7097
Epoch 2/10
4096/4096 [==============================] - 109s 27ms/step - loss: 0.1707 - acc: 0.7596 - val_loss: 0.1719 - val_acc: 0.7474
Epoch 3/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.1586 - acc: 0.7774 - val_loss: 0.1675 - val_acc: 0.7527
Epoch 4/10
4096/4096 [==============================] - 88s 22ms/step - loss: 0.1559 - acc: 0.7816 - val_loss: 0.1665 - val_acc: 0.7539
Epoch 5/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.1539 - acc: 0.7844 - val_loss: 0.1628 - val_acc: 0.7583
Epoch 6/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.1523 - acc: 0.7867 - val_loss: 0.1661 - val_acc: 0.7535
Epoch 7/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.1509 - acc: 0.7889 - val_loss: 0.1590 - val_acc: 0.7634
Epoch 8/10
4096/4096 [=========================

In [74]:
history_1_2_3_version_2= model_1_2_3_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


Epoch 1/10
4096/4096 [==============================] - 92s 22ms/step - loss: 0.1356 - acc: 0.8106 - val_loss: 0.1363 - val_acc: 0.8116
Epoch 2/10
4096/4096 [==============================] - 109s 27ms/step - loss: 0.0973 - acc: 0.8693 - val_loss: 0.1259 - val_acc: 0.8242
Epoch 3/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.0850 - acc: 0.8882 - val_loss: 0.1356 - val_acc: 0.8084
Epoch 4/10
4096/4096 [==============================] - 88s 22ms/step - loss: 0.0789 - acc: 0.8965 - val_loss: 0.1680 - val_acc: 0.7815
Epoch 5/10
4096/4096 [==============================] - 88s 22ms/step - loss: 0.0753 - acc: 0.9021 - val_loss: 0.1016 - val_acc: 0.8608
Epoch 6/10
4096/4096 [==============================] - 88s 22ms/step - loss: 0.0738 - acc: 0.9050 - val_loss: 0.1116 - val_acc: 0.8572
Epoch 7/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.0722 - acc: 0.9075 - val_loss: 0.1144 - val_acc: 0.8481
Epoch 8/10
4096/4096 [=========================

In [79]:
# Building the Convolutonal Nueral Network Model 4.01 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.1)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.1)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = hinge
model_4_01_version_2 = Model(inputs=input, outputs = output)
model_4_01_version_2.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['acc'])

model_4_01_version_2.summary()
#plot_model(model_4_01_version_2)

Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_171 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_172 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_87 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_173 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_174 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [83]:
model_4_01_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

Epoch 1/10
4096/4096 [==============================] - 91s 22ms/step - loss: 0.4036 - acc: 0.8184 - val_loss: 0.3973 - val_acc: 0.8219
Epoch 2/10
4096/4096 [==============================] - 108s 26ms/step - loss: 0.2863 - acc: 0.8813 - val_loss: 0.4561 - val_acc: 0.8055
Epoch 3/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2457 - acc: 0.9015 - val_loss: 0.4408 - val_acc: 0.8149
Epoch 4/10
4096/4096 [==============================] - 87s 21ms/step - loss: 0.2222 - acc: 0.9126 - val_loss: 0.4171 - val_acc: 0.8263
Epoch 5/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.2074 - acc: 0.9189 - val_loss: 0.3814 - val_acc: 0.8548
Epoch 6/10
4096/4096 [==============================] - 87s 21ms/step - loss: 0.1936 - acc: 0.9251 - val_loss: 0.3808 - val_acc: 0.8483
Epoch 7/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.1831 - acc: 0.9297 - val_loss: 0.3622 - val_acc: 0.8572
Epoch 8/10
4096/4096 [=========================

In [84]:
# Building the Convolutonal Nueral Network Model 4.06 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.6)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.6)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = hinge
model_4_06_version_2 = Model(inputs=input, outputs = output)
model_4_06_version_2.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['acc'])

model_4_06_version_2.summary()
#plot_model(model_4_06_version_2)

Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_177 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_178 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_90 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_179 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_180 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [85]:
model_4_06_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

Epoch 1/10
4096/4096 [==============================] - 92s 22ms/step - loss: 0.4082 - acc: 0.8184 - val_loss: 0.4099 - val_acc: 0.8192
Epoch 2/10
4096/4096 [==============================] - 109s 27ms/step - loss: 0.3064 - acc: 0.8736 - val_loss: 0.3963 - val_acc: 0.8274
Epoch 3/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2700 - acc: 0.8916 - val_loss: 0.3963 - val_acc: 0.8248
Epoch 4/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.2485 - acc: 0.9019 - val_loss: 0.3414 - val_acc: 0.8533
Epoch 5/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2322 - acc: 0.9098 - val_loss: 0.3506 - val_acc: 0.8562
Epoch 6/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2223 - acc: 0.9147 - val_loss: 0.3565 - val_acc: 0.8574
Epoch 7/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2131 - acc: 0.9188 - val_loss: 0.3579 - val_acc: 0.8600
Epoch 8/10
4096/4096 [=========================

In [86]:
# Building the Convolutonal Nueral Network Model 4.07 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = hinge
model_4_07_version_2 = Model(inputs=input, outputs = output)
model_4_07_version_2.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['acc'])

model_4_07_version_2.summary()
#plot_model(model_4_07_version_2)

Model: "model_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_32 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_183 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_184 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_93 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_185 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_186 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [87]:
model_4_07_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

Epoch 1/10
4096/4096 [==============================] - 92s 22ms/step - loss: 0.4179 - acc: 0.8124 - val_loss: 0.4206 - val_acc: 0.8098
Epoch 2/10
4096/4096 [==============================] - 87s 21ms/step - loss: 0.3076 - acc: 0.8726 - val_loss: 0.3745 - val_acc: 0.8363
Epoch 3/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.2695 - acc: 0.8907 - val_loss: 0.4356 - val_acc: 0.8116
Epoch 4/10
4096/4096 [==============================] - 110s 27ms/step - loss: 0.2460 - acc: 0.9028 - val_loss: 0.3667 - val_acc: 0.8464
Epoch 5/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2293 - acc: 0.9107 - val_loss: 0.3843 - val_acc: 0.8504
Epoch 6/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2193 - acc: 0.9149 - val_loss: 0.4000 - val_acc: 0.8413
Epoch 7/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2086 - acc: 0.9192 - val_loss: 0.4053 - val_acc: 0.8347
Epoch 8/10
4096/4096 [=========================

In [88]:
# Building the Convolutonal Nueral Network Model 4.08 (Version 2)

# Setting up the output with the right size
input = Input(shape=(96,96,3))

# Relu activation functions only, with softmax activation function for the last Dense layer
x = Conv2D(16, (3,3), activation='relu', padding='valid')(input)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
x = Conv2D(16, (3,3), activation='relu', padding='valid')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.)(x)
output = Dense(2, activation='softmax')(x)

# Optimizer = adam
# Loss function = hinge
model_4_08_version_2 = Model(inputs=input, outputs = output)
model_4_08_version_2.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['acc'])

model_4_08_version_2.summary()
#plot_model(model_4_08_version_2)

Model: "model_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_33 (InputLayer)       [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_189 (Conv2D)         (None, 94, 94, 16)        448       
                                                                 
 conv2d_190 (Conv2D)         (None, 92, 92, 16)        2320      
                                                                 
 max_pooling2d_96 (MaxPoolin  (None, 46, 46, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_191 (Conv2D)         (None, 44, 44, 32)        4640      
                                                                 
 conv2d_192 (Conv2D)         (None, 42, 42, 16)        4624      
                                                          

In [89]:
model_4_08_version_2.fit(train_ds, validation_data=validation_ds, epochs = 10, verbose = 1, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

Epoch 1/10
4096/4096 [==============================] - 91s 22ms/step - loss: 0.4032 - acc: 0.8208 - val_loss: 0.3601 - val_acc: 0.8377
Epoch 2/10
4096/4096 [==============================] - 109s 27ms/step - loss: 0.2977 - acc: 0.8768 - val_loss: 0.3853 - val_acc: 0.8320
Epoch 3/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2593 - acc: 0.8946 - val_loss: 0.3243 - val_acc: 0.8591
Epoch 4/10
4096/4096 [==============================] - 88s 22ms/step - loss: 0.2385 - acc: 0.9054 - val_loss: 0.3928 - val_acc: 0.8290
Epoch 5/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2228 - acc: 0.9120 - val_loss: 0.3949 - val_acc: 0.8432
Epoch 6/10
4096/4096 [==============================] - 88s 21ms/step - loss: 0.2112 - acc: 0.9178 - val_loss: 0.3455 - val_acc: 0.8519
Epoch 7/10
4096/4096 [==============================] - 89s 22ms/step - loss: 0.2028 - acc: 0.9214 - val_loss: 0.4486 - val_acc: 0.8264
Epoch 8/10
4096/4096 [=========================

Saving the models

In [28]:
import joblib

# Models 1, 2, 3
joblib.dump(model_1_version_2, 'model_1_version_2')
joblib.dump(model_2_version_2, 'model_2_version_2')
joblib.dump(model_3_version_2, 'model_3_version_2')

INFO:tensorflow:Assets written to: ram://a6818ad0-b7ea-41c9-9465-16f9e09f4f7b/assets


INFO:tensorflow:Assets written to: ram://a6818ad0-b7ea-41c9-9465-16f9e09f4f7b/assets


['model_1_version_2']

In [75]:
# Models 1.1., 1.2, 2.1, 2.2, 3.1, 3.2
joblib.dump(model_1_1_version_2, 'model_1_1_version_2')
joblib.dump(model_1_2_version_2, 'model_1_2_version_2')
joblib.dump(model_2_1_version_2, 'model_2_1_version_2')
joblib.dump(model_2_2_version_2, 'model_2_2_version_2')
joblib.dump(model_3_1_version_2, 'model_3_1_version_2')
joblib.dump(model_3_2_version_2, 'model_3_2_version_2')

INFO:tensorflow:Assets written to: ram://248fc3e2-aecb-4b54-ad36-1e1732ec2abb/assets


INFO:tensorflow:Assets written to: ram://248fc3e2-aecb-4b54-ad36-1e1732ec2abb/assets


INFO:tensorflow:Assets written to: ram://2201679e-44f1-4542-9fca-d7c68ba1df77/assets


INFO:tensorflow:Assets written to: ram://2201679e-44f1-4542-9fca-d7c68ba1df77/assets


INFO:tensorflow:Assets written to: ram://d24223d0-4985-4d56-b75c-b0bf0d751394/assets


...

INFO:tensorflow:Assets written to: ram://a18668d6-fba0-40fe-b1d4-3eb5e7e93c55/assets


INFO:tensorflow:Assets written to: ram://a18668d6-fba0-40fe-b1d4-3eb5e7e93c55/assets


INFO:tensorflow:Assets written to: ram://76beb2e6-02a1-4b21-be5b-888b8aaa600d/assets


INFO:tensorflow:Assets written to: ram://76beb2e6-02a1-4b21-be5b-888b8aaa600d/assets


['model_3_2_version_2']

In [76]:
#Models 1.1.1, 1.1.2, 1.1.3, 1.2.1, 1.2.2, 1.2.3, 2.1.1, 2.1.2, 2.1.3, 2.2.1, 2.2.2, 2.2.3, 3.1.1, 3.1.2, 3.1.3, 3.2.1, 3.2.2, 3.2.3.

joblib.dump(model_1_1_1_version_2, 'model_1_1_1_version_2')
joblib.dump(model_1_1_2_version_2, 'model_1_1_2_version_2')
joblib.dump(model_1_1_3_version_2, 'model_1_1_3_version_2')

joblib.dump(model_1_2_1_version_2, 'model_1_2_1_version_2')
joblib.dump(model_1_2_2_version_2, 'model_1_2_2_version_2')
joblib.dump(model_1_2_3_version_2, 'model_1_2_3_version_2')

joblib.dump(model_2_1_1_version_2, 'model_2_1_1_version_2')
joblib.dump(model_2_1_2_version_2, 'model_2_1_2_version_2')
joblib.dump(model_2_1_3_version_2, 'model_2_1_3_version_2')

joblib.dump(model_2_2_1_version_2, 'model_2_2_1_version_2')
joblib.dump(model_2_2_2_version_2, 'model_2_2_2_version_2')
joblib.dump(model_2_2_3_version_2, 'model_2_2_3_version_2')

joblib.dump(model_3_1_1_version_2, 'model_3_1_1_version_2')
joblib.dump(model_3_1_2_version_2, 'model_3_1_2_version_2')
joblib.dump(model_3_1_3_version_2, 'model_3_1_3_version_2')

joblib.dump(model_3_2_1_version_2, 'model_3_2_1_version_2')
joblib.dump(model_3_2_2_version_2, 'model_3_2_2_version_2')
joblib.dump(model_3_2_3_version_2, 'model_3_2_3_version_2')

INFO:tensorflow:Assets written to: ram://2f3d10b1-ff60-45e6-b08a-2da433e99109/assets


INFO:tensorflow:Assets written to: ram://2f3d10b1-ff60-45e6-b08a-2da433e99109/assets


INFO:tensorflow:Assets written to: ram://2e205d7c-20c8-45a5-ad46-4ef79eefb639/assets


INFO:tensorflow:Assets written to: ram://2e205d7c-20c8-45a5-ad46-4ef79eefb639/assets


INFO:tensorflow:Assets written to: ram://a589e7d7-8ac9-417b-8bb9-4ea873f560e6/assets


...

INFO:tensorflow:Assets written to: ram://21daaa96-7b30-4283-b2c5-c95a28d694e2/assets


INFO:tensorflow:Assets written to: ram://21daaa96-7b30-4283-b2c5-c95a28d694e2/assets


INFO:tensorflow:Assets written to: ram://23365eca-06e7-4c99-8417-bab0c2467735/assets


INFO:tensorflow:Assets written to: ram://23365eca-06e7-4c99-8417-bab0c2467735/assets


['model_3_2_3_version_2']

In [80]:
joblib.dump(model_4_01_version_2, 'model_4_01_version_2')

INFO:tensorflow:Assets written to: ram://395754e7-a043-4aa0-9ec4-e9aed5d0ebfa/assets


INFO:tensorflow:Assets written to: ram://395754e7-a043-4aa0-9ec4-e9aed5d0ebfa/assets


['model_4_01_version_2']

In [93]:
joblib.dump(model_4_06_version_2, 'model_4_06_version_2')

INFO:tensorflow:Assets written to: ram://46424e75-fa97-47ee-a2be-2cdffab90217/assets


INFO:tensorflow:Assets written to: ram://46424e75-fa97-47ee-a2be-2cdffab90217/assets


['model_4_06_version_2']

In [94]:
joblib.dump(model_4_07_version_2, 'model_4_07_version_2')

INFO:tensorflow:Assets written to: ram://6cc32f53-1f64-4d5a-9079-39964860cb12/assets


INFO:tensorflow:Assets written to: ram://6cc32f53-1f64-4d5a-9079-39964860cb12/assets


['model_4_07_version_2']

In [95]:
joblib.dump(model_4_08_version_2, 'model_4_08_version_2')

INFO:tensorflow:Assets written to: ram://dfd0ffda-9b27-428e-b165-1c0fe02b513a/assets


INFO:tensorflow:Assets written to: ram://dfd0ffda-9b27-428e-b165-1c0fe02b513a/assets


['model_4_08_version_2']

Loading the saved models

In [43]:
# Models 1, 2, 3
model_1_version_2_object = joblib.load('model_1_version_2')
model_2_version_2_object = joblib.load('model_2_version_2')
model_3_version_2_object = joblib.load('model_3_version_2')

In [38]:

# Models 1.1., 1.2, 2.1, 2.2, 3.1, 3.2
model_1_1_version_2_object = joblib.load('model_1_1_version_2')
model_1_2_version_2_object = joblib.load('model_1_2_version_2')
model_2_1_version_2_object = joblib.load('model_2_1_version_2')
model_2_2_version_2_object = joblib.load('model_2_2_version_2')
model_3_1_version_2_object = joblib.load('model_3_1_version_2')
model_3_2_version_2_object = joblib.load('model_3_2_version_2')

In [77]:
#Models 1.1.1, 1.1.2, 1.1.3, 1.2.1, 1.2.2, 1.2.3, 2.1.1, 2.1.2, 2.1.3, 2.2.1, 2.2.2, 2.2.3, 3.1.1, 3.1.2, 3.1.3, 3.2.1, 3.2.2, 3.2.3.

model_1_1_1_version_2_object= joblib.load('model_1_1_1_version_2')
model_1_1_2_version_2_object= joblib.load('model_1_1_2_version_2')
model_1_1_3_version_2_object= joblib.load('model_1_1_3_version_2')

model_1_2_1_version_2_object= joblib.load('model_1_2_1_version_2')
model_1_2_2_version_2_object= joblib.load('model_1_2_2_version_2')
model_1_2_3_version_2_object= joblib.load('model_1_2_3_version_2')

model_2_1_1_version_2_object= joblib.load('model_2_1_1_version_2')
model_2_1_2_version_2_object= joblib.load('model_2_1_2_version_2')
model_2_1_3_version_2_object= joblib.load('model_2_1_3_version_2')

model_2_2_1_version_2_object= joblib.load('model_2_2_1_version_2')
model_2_2_2_version_2_object= joblib.load('model_2_2_2_version_2')
model_2_2_3_version_2_object= joblib.load('model_2_2_3_version_2')

model_3_1_1_version_2_object= joblib.load('model_3_1_1_version_2')
model_3_1_2_version_2_object= joblib.load('model_3_1_2_version_2')
model_3_1_3_version_2_object= joblib.load('model_3_1_3_version_2')

model_3_2_1_version_2_object= joblib.load('model_3_2_1_version_2')
model_3_2_2_version_2_object= joblib.load('model_3_2_2_version_2')
model_3_2_3_version_2_object= joblib.load('model_3_2_3_version_2')

In [81]:
model_4_01_version_2_object = joblib.load('model_4_01_version_2')

In [96]:
model_4_06_version_2_object = joblib.load('model_4_06_version_2')

In [97]:
model_4_07_version_2_object = joblib.load('model_4_07_version_2')

In [98]:
model_4_08_version_2_object = joblib.load('model_4_08_version_2')

Evaluating models

In [30]:
# Model 1, 2, 3

print("Test dataset accuracy for model 1 version 2 is: {0:.4f}".format(model_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 2 version 2 is: {0:.4f}".format(model_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 3 version 2 is: {0:.4f}".format(model_3_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

256/256 [==============================] - 8s 31ms/step - loss: 0.4693 - acc: 0.8337
Test dataset accuracy for model 1 version 2 is: 0.8337
256/256 [==============================] - 8s 31ms/step - loss: 0.6988 - acc: 0.4998
Test dataset accuracy for model 2 version 2 is: 0.4998
256/256 [==============================] - 8s 29ms/step - loss: 0.6963 - acc: 0.4998
Test dataset accuracy for model 3 version 2 is: 0.4998


In [44]:
# Models 1.1., 1.2, 2.1, 2.2, 3.1, 3.2


print("Test dataset accuracy for model 1.1 version 2 is: {0:.4f}".format(model_1_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 1.2 version 2 is: {0:.4f}".format(model_1_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

print("Test dataset accuracy for model 2.1 version 2 is: {0:.4f}".format(model_2_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 1.2 version 2 is: {0:.4f}".format(model_2_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

print("Test dataset accuracy for model 3.1 version 2 is: {0:.4f}".format(model_3_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 3.2 version 2 is: {0:.4f}".format(model_3_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))


256/256 [==============================] - 8s 30ms/step - loss: 1.0017 - acc: 0.4998
Test dataset accuracy for model 1.1 version 2 is: 0.4998
256/256 [==============================] - 8s 32ms/step - loss: 0.2497 - acc: 0.5002
Test dataset accuracy for model 1.2 version 2 is: 0.5002
256/256 [==============================] - 8s 32ms/step - loss: 0.2497 - acc: 0.5002
Test dataset accuracy for model 1.2 version 2 is: 0.5002
256/256 [==============================] - 8s 32ms/step - loss: 0.2497 - acc: 0.5002
Test dataset accuracy for model 1.2 version 2 is: 0.5002
256/256 [==============================] - 8s 33ms/step - loss: 0.2497 - acc: 0.5002
Test dataset accuracy for model 1.2 version 2 is: 0.5002
256/256 [==============================] - 8s 33ms/step - loss: 0.2497 - acc: 0.5002
Test dataset accuracy for model 1.2 version 2 is: 0.5002


In [78]:
# Model 1.1.1, 1.1.2, 1.1.3, 1.2.1, 1.2.2, 1.2.3, 2.1.1, 2.1.2, 2.1.3, 2.2.1, 2.2.2, 2.2.3, 3.1.1, 3.1.2, 3.1.3, 3.2.1, 3.2.2, 3.2.3

print("Test dataset accuracy for model 1.1.1 version 2 is: {0:.4f}".format(model_1_1_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 1.1.2 version 2 is: {0:.4f}".format(model_1_1_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 1.1.3 version 2 is: {0:.4f}".format(model_1_1_3_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

print("Test dataset accuracy for model 1.2.1 version 2 is: {0:.4f}".format(model_1_2_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 1.2.2 version 2 is: {0:.4f}".format(model_1_2_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 1.2.3 version 2 is: {0:.4f}".format(model_1_2_3_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

print("Test dataset accuracy for model 2.1.1 version 2 is: {0:.4f}".format(model_2_1_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 2.1.2 version 2 is: {0:.4f}".format(model_2_1_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 2.1.3 version 2 is: {0:.4f}".format(model_2_1_3_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

print("Test dataset accuracy for model 2.2.1 version 2 is: {0:.4f}".format(model_2_2_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 2.2.2 version 2 is: {0:.4f}".format(model_2_2_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 2.2.3 version 2 is: {0:.4f}".format(model_2_2_3_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

print("Test dataset accuracy for model 3.1.1 version 2 is: {0:.4f}".format(model_3_1_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 3.1.2 version 2 is: {0:.4f}".format(model_3_1_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 3.1.3 version 2 is: {0:.4f}".format(model_3_1_3_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

print("Test dataset accuracy for model 3.2.1 version 2 is: {0:.4f}".format(model_3_2_1_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 3.2.2 version 2 is: {0:.4f}".format(model_3_2_2_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))
print("Test dataset accuracy for model 3.2.3 version 2 is: {0:.4f}".format(model_3_2_3_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

256/256 [==============================] - 8s 31ms/step - loss: 0.6757 - acc: 0.8244
Test dataset accuracy for model 1.1.1 version 2 is: 0.8244
256/256 [==============================] - 8s 29ms/step - loss: 0.7736 - acc: 0.7270
Test dataset accuracy for model 1.1.2 version 2 is: 0.7270
256/256 [==============================] - 8s 31ms/step - loss: 1.0005 - acc: 0.4016
Test dataset accuracy for model 1.1.3 version 2 is: 0.4016
256/256 [==============================] - 8s 31ms/step - loss: 0.1524 - acc: 0.7840
Test dataset accuracy for model 1.2.1 version 2 is: 0.7840
256/256 [==============================] - 8s 31ms/step - loss: 0.1631 - acc: 0.7670
Test dataset accuracy for model 1.2.2 version 2 is: 0.7670
256/256 [==============================] - 8s 31ms/step - loss: 0.1331 - acc: 0.8386
Test dataset accuracy for model 1.2.3 version 2 is: 0.8386
256/256 [==============================] - 8s 31ms/step - loss: 1.0000 - acc: 0.5002
Test dataset accuracy for model 2.1.1 version 2 is:

In [82]:
print("Test dataset accuracy for model 4.01 version 2 is: {0:.4f}".format(model_4_01_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

256/256 [==============================] - 8s 30ms/step - loss: 0.6937 - acc: 0.4962
Test dataset accuracy for model 4.01 version 2 is: 0.4962


In [99]:
print("Test dataset accuracy for model 4.06 version 2 is: {0:.4f}".format(model_4_06_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

256/256 [==============================] - 8s 31ms/step - loss: 0.5021 - acc: 0.8087
Test dataset accuracy for model 4.06 version 2 is: 0.8087


In [100]:
print("Test dataset accuracy for model 4.07 version 2 is: {0:.4f}".format(model_4_07_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

256/256 [==============================] - 8s 31ms/step - loss: 0.5330 - acc: 0.7910
Test dataset accuracy for model 4.07 version 2 is: 0.7910


In [101]:
print("Test dataset accuracy for model 4.08 version 2 is: {0:.4f}".format(model_4_08_version_2_object.evaluate(test_ds, steps=steps, verbose=1)[1]))

256/256 [==============================] - 8s 31ms/step - loss: 0.5602 - acc: 0.8125
Test dataset accuracy for model 4.08 version 2 is: 0.8125
